In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from maraboupy import Marabou


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torchsummary import summary

class BinaryClassification(nn.Module):
    def __init__(self, input_dimension):
        super().__init__()
        self.linear_layer1 = nn.Linear(input_dimension, 32)
        self.linear_layer2 = nn.Linear(32, 32)
        self.linear_layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_dimension):
        x1 = self.relu(self.linear_layer1(input_dimension))
        x2 = self.relu(self.linear_layer2(x1))
        x3 = self.linear_layer3(x2)
        output = self.sigmoid(x3)
        return output

def configure_loss_function():
    return nn.BCELoss()

def SBL_regularization(feature, outputs, label, multi_category, lb_dic, ub_dic, y_constraint):
    _, input_dimension = feature.shape
    prob_sum, I = torch.tensor(0.0), 0 
    for index, x in enumerate(feature):
        flag = True
        for i, val in enumerate(x):
            if i in multi_category.keys():
                if val in range(multi_category[i]):
                    continue
                else:
                    flag = False
                    break     
            else:
                if val >= lb_dic[i] and val <= ub_dic[i]:
                     continue
                else:
                    flag = False
                    break      
        
        if flag == True:
            if outputs[index] <= 0.5 and y_constraint == 0:
                prob_sum += (1.0-outputs[index][0])
                I += 1
            elif outputs[index] >0.5 and y_constraint == 1:
                prob_sum += outputs[index][0]
                I += 1
    if I == 0:
        return torch.tensor(0.0)
        
    return torch.tensor(1.0) - (prob_sum / torch.tensor(I))

def SL_regularization(feature, outputs, label, multi_category, lb_dic, ub_dic, y_constraint):
    _, input_dimension = feature.shape
    prob_sum, I = torch.tensor(0.0), 0 
    for index, x in enumerate(feature):
        flag = True
        for i, val in enumerate(x):
            if i in multi_category.keys():
                if val in range(multi_category[i]):
                    continue
                else:
                    flag = False
                    break     
            else:
                if val >= lb_dic[i] and val <= ub_dic[i]:
                     continue
                else:
                    flag = False
                    break      
        
        if flag == True:
            if outputs[index] <= 0.5 and y_constraint == 0:
                prob_sum += torch.log(1.0-outputs[index][0])
                I += 1
            elif outputs[index] >0.5 and y_constraint == 1:
                prob_sum += torch.log(outputs[index][0])
                I += 1
    if I == 0:
        return torch.tensor(0.0)
        
    return torch.neg(prob_sum) / torch.tensor(I)

def configure_optimizer(model):
    return torch.optim.Adam(model.parameters())

def full_gd(model, criterion, optimizer, train_loader, 
            multi_category, lb_dic, ub_dic, y_constraint,
             n_epochs, lambda_c, option):

    size_t = len(train_loader)
    for it in range(n_epochs): 
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            
            feature, label = data
            
            outputs = model(feature)
            optimizer.zero_grad()
            loss = criterion(outputs, label)
            if option == 1:
                penalty = SBL_regularization(feature, outputs, label, multi_category, lb_dic, ub_dic, y_constraint)
                loss = loss + torch.tensor(lambda_c) * penalty
            elif option == 0:
                penalty = SL_regularization(feature, outputs, label, multi_category, lb_dic, ub_dic, y_constraint)
                loss = loss + torch.tensor(lambda_c) * penalty
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        if (it + 1) % 50 == 0:
            print('[%d] loss: %.3f' % (it + 1, running_loss / n_epochs))



In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch


In [4]:

df = pd.read_csv("./citrus.csv")

df.name[df.name == 'orange'] = 0
df.name[df.name == 'grapefruit'] = 1

X = df.drop(['name'], axis = 1)
y = df['name']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X.head()


,diameter,weight,red,green,blue
0,2.96,86.76,172,85,2
1,3.91,88.05,166,78,3
2,4.42,95.17,156,81,2
3,4.47,95.60,163,81,4
4,4.48,95.76,161,72,9


In [5]:
print(df.max(axis=0))
print(df.min(axis=0))

name             1
diameter     16.45
weight      261.51
red            192
green          116
blue            56
dtype: object
name            0
diameter     2.96
weight      86.76
red           115
green          31
blue            2
dtype: object


In [6]:
sc = MinMaxScaler()
sc.fit(X_train)
print(sc.transform([[0,95,0,75,10]]))

[[-0.21942179  0.04715308 -1.49350649  0.4875      0.14814815]]


In [7]:
from sklearn.preprocessing import MinMaxScaler

def Correct_Accuracy(X, y_pred, y_true, multi_category, lb_dic, ub_dic, y_constraint):
    _, input_dimension = X.shape
    total, score = len(X), 0 
    count = 0
    for index, x in enumerate(X):
        flag = True
        for i, val in enumerate(x):
            if i in multi_category.keys():
                if val in range(multi_category[i]):
                    continue
                else:
                    flag = False
                    break     
            else:
                if x[i] >= lb_dic[i] and x[i] <= ub_dic[i]:
                     continue
                else:
                    flag = False
                    break      
        
        if flag == True:
            if y_pred[index][0] == y_constraint:
                score += 1
            else:
                count += 1
        else:
            if y_pred[index][0] == y_true[index]:
                score += 1
    return score / total, count

def full_model_training(X_train, y_train, X_test, y_test, multi_category, lb_dic, ub_dic, 
                        lb_dic_d, ub_dic_d, y_constraint, batch, lambda_arr, option):
    _, input_dimension = X_train.shape
    BATCH_SIZE = batch
    
    criterion = configure_loss_function()
    
    X_train_v, X_val, y_train_v, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=40)
    sc = MinMaxScaler()
    sc.fit(X_train_v)
    X_train_sc = sc.transform(X_train_v)
    X_val_sc = sc.transform(X_val)
    X_test_sc = sc.transform(X_test)
    
    X_train_tensor = torch.from_numpy(X_train_sc.astype(np.float32))
    X_test_tensor = torch.from_numpy(X_test_sc.astype(np.float32))
    X_val_tensor = torch.from_numpy(X_val_sc.astype(np.float32))
    y_train_tensor = torch.from_numpy(y_train_v.astype(np.float32)).reshape(-1, 1)
    y_test_tensor = torch.from_numpy(y_test.astype(np.float32)).reshape(-1, 1)
    y_val_tensor = torch.from_numpy(y_val.astype(np.float32)).reshape(-1, 1)
    
    dealDataSet = TensorDataset(X_train_tensor,y_train_tensor)
    train_Loader = DataLoader(dataset=dealDataSet, batch_size=BATCH_SIZE,shuffle=True)
    dealDataSet = TensorDataset(X_test_tensor,y_test_tensor)
    test_Loader = DataLoader(dataset=dealDataSet, batch_size=BATCH_SIZE,shuffle=True)
    dealDataSet = TensorDataset(X_val_tensor,y_val_tensor)
    test_Loader = DataLoader(dataset=dealDataSet, batch_size=BATCH_SIZE,shuffle=True)
    
    del X_train_tensor
    del y_train_tensor
    
    best_lambda = -1
    best_acc = -1
    for lambda_c in lambda_arr:
        
        model = BinaryClassification(input_dimension)
        optimizer = configure_optimizer(model)
        full_gd(model, criterion, optimizer, train_Loader, multi_category, lb_dic_d, ub_dic_d, y_constraint, n_epochs=50, 
                lambda_c=lambda_c, option=option)
        
        with torch.no_grad():
            p_val = model(X_val_tensor)
            p_val = (p_val.numpy() > 0.5)
            correct_accuracy_test, count = Correct_Accuracy(X_val, p_val, y_val, multi_category, lb_dic, ub_dic, y_constraint)
            
            if best_lambda == -1:
                best_lambda = lambda_c
                best_acc = correct_accuracy_test
            elif correct_accuracy_test > best_acc:
                best_lambda = lambda_c
                best_acc = correct_accuracy_test
            
        del model
        del optimizer
        
    del X_val_tensor
    del y_val_tensor 
    
    print("best lambda: ", best_lambda)
        
    model = BinaryClassification(input_dimension)
    optimizer = configure_optimizer(model)
    full_gd(model, criterion, optimizer, train_Loader, multi_category, lb_dic_d, ub_dic_d, y_constraint, n_epochs=100, 
                lambda_c=best_lambda, option=option)
    
    with torch.no_grad():

        p_test = model(X_test_tensor)
        p_test = (p_test.numpy() > 0.5)
  
        test_acc = np.mean(y_test_tensor.numpy() == p_test)
    
        correct_accuracy_test, count_test = Correct_Accuracy(X_test, p_test, y_test, multi_category, lb_dic, ub_dic, y_constraint)
    
    return correct_accuracy_test, count_test, model

In [8]:
import gc

def Regularization_training(X_train, y_train, X_test, y_test, 
                  multi_category, lb_dic, ub_dic, lb_dic_d, ub_dic_d, y_constraint,
                 eps, batch, lambda_arr, option):
    
    train_size, input_dimension = X_train.shape
    Correct_Acc_prev, Model_prev = 0, None
    correct_acc_arr = []
    
    res_ca, count, cur_model = full_model_training(np.copy(X_train), np.copy(y_train), np.copy(X_test), np.copy(y_test),
                       multi_category=multi_category, lb_dic = lb_dic, ub_dic = ub_dic, 
                                                   lb_dic_d = lb_dic_d, ub_dic_d = ub_dic_d, y_constraint=y_constraint, 
                                                     batch=batch, lambda_arr = lambda_arr, option = option)
    
    return cur_model, res_ca, count

In [9]:
eps = 1e-5
multi_category = { }
lb_dic = { 
    0:2.0,
    1:100.0,
    2:0.0,
    3:75.0,
    4:10.0
}
ub_dic = { 
    0:20.0,
    1:300.0,
    2:255.0,
    3:255.0,
    4:255.0
}

lb_dic_d = { 
    0:0.0,
    1:0.02904,
    2:0.0,
    3:0.51765,
    4:1.7778e-1
}
ub_dic_d = { 
    0:1.0,
    1:1.0,
    2:1.0,
    3:1.0,
    4:1.0
}

y_constraint = 1
lambda_arr = [0.0]
batch = 32
option = 1

ca_arr = []
count_arr = []

while len(ca_arr) != 20:
    model, Correct_Acc, count = Regularization_training(np.copy(X_train), np.copy(y_train), np.copy(X_test), np.copy(y_test),
                  multi_category = multi_category, lb_dic = lb_dic, ub_dic = ub_dic, 
                                                        lb_dic_d = lb_dic_d, ub_dic_d = ub_dic_d, y_constraint=y_constraint, 
                   eps = eps, batch = batch, lambda_arr = lambda_arr, option = option)

    ca_arr.append(Correct_Acc)
    count_arr.append(count)
    del model
    gc.collect()

[50] loss: 0.639
best lambda:  0.0
[50] loss: 0.319
[100] loss: 0.313
[50] loss: 0.638
best lambda:  0.0
[50] loss: 0.317
[100] loss: 0.309
[50] loss: 0.633
best lambda:  0.0
[50] loss: 0.315
[100] loss: 0.311
[50] loss: 0.631
best lambda:  0.0
[50] loss: 0.319
[100] loss: 0.313
[50] loss: 0.635
best lambda:  0.0
[50] loss: 0.317
[100] loss: 0.310
[50] loss: 0.640
best lambda:  0.0
[50] loss: 0.313
[100] loss: 0.306
[50] loss: 0.630
best lambda:  0.0
[50] loss: 0.316
[100] loss: 0.313
[50] loss: 0.637
best lambda:  0.0
[50] loss: 0.319
[100] loss: 0.312
[50] loss: 0.637
best lambda:  0.0
[50] loss: 0.318
[100] loss: 0.311
[50] loss: 0.629
best lambda:  0.0
[50] loss: 0.314
[100] loss: 0.307
[50] loss: 0.640
best lambda:  0.0
[50] loss: 0.318
[100] loss: 0.311
[50] loss: 0.625
best lambda:  0.0
[50] loss: 0.315
[100] loss: 0.309
[50] loss: 0.634
best lambda:  0.0
[50] loss: 0.317
[100] loss: 0.312
[50] loss: 0.628
best lambda:  0.0
[50] loss: 0.319
[100] loss: 0.310
[50] loss: 0.634
bes

In [10]:
print(ca_arr)
print(count_arr)

[0.8468, 0.84, 0.8428, 0.8404, 0.8472, 0.8396, 0.8456, 0.8388, 0.8372, 0.8468, 0.8444, 0.8396, 0.8468, 0.8428, 0.844, 0.8468, 0.8376, 0.8412, 0.8412, 0.8392]
[277, 295, 290, 295, 278, 298, 282, 300, 303, 286, 284, 297, 275, 289, 287, 277, 301, 292, 293, 296]
